In [1]:
from typing import List
Image = List[List[str]]

In [2]:
def read_data(path):
    f = open(path,"r")
    lines = [line.strip() for line in f.readlines()]
    image = [[i for i in line] for line in lines[2:]]
    return (lines[0], image)

In [3]:
demo_dataset = read_data("demo.txt")
full_dataset = read_data("data.txt")
demo_dataset

('..#.#..#####.#.#.#.###.##.....###.##.#..###.####..#####..#....#..#..##..###..######.###...####..#..#####..##..#.#####...##.#.#..#.##..#.#......#.###.######.###.####...#.##.##..#..#..#####.....#.#....###..#.##......#.....#..#..#..##..#...##.######.####.####.#.#...#.......#..#.#.#...####.##.#......#..#...##.#.##..#...##.#.##..###.#......#.#.......#.#.#.####.###.##...#.....####.#..#..#.##.#....##..#.####....##...##..#...#......#.#.......#.......##..####..#...#.#.#...##..#.#..###..#####........#..####......#..#',
 [['#', '.', '.', '#', '.'],
  ['#', '.', '.', '.', '.'],
  ['#', '#', '.', '.', '#'],
  ['.', '.', '#', '.', '.'],
  ['.', '.', '#', '#', '#']])

# Part 1

In [4]:
def to_bit(character: str) -> int:
    return 1 if character == "#" else 0

In [5]:
from functools import lru_cache

@lru_cache(512)
def convert_binary(string: str) -> int:
    exp = len(string) - 1
    value = 0
    for i in string:
        value += to_bit(i) * 2**exp
        exp -= 1
    return value

In [6]:
assert convert_binary("...#...#.") == 34

In [7]:
def count_light(image: List[List[str]]) -> int:
    return sum([to_bit(i) for row in image for i in row])

In [8]:
assert count_light(demo_dataset[1]) == 10

In [9]:
def get_value(image: Image, i: int, j: int, fill: str) -> str:
    if (i<0) or (i >= len(image)):
        return fill
    else:
        row = image[i]
        if (j<0) or (j >= len(row)):
            return fill
        else:
            return row[j]

In [10]:
def get_binary_pixel(image, i: int, j: int, fill: str) -> str:
    characters = [get_value(image, x, y, fill) for x in [i - 1, i, i +1] for y in [j- 1, j, j+1]]
    return "".join(characters)

In [11]:
assert get_binary_pixel(demo_dataset[1], 2, 2, ".") == "...#...#."

In [12]:
def empty_row(length: int, value: str) -> List[str]:
    return [value for i in range(length)]

In [13]:
def increase_image_size(image: Image, fill: str = ".") -> Image:
    row_size = len(image[0]) + 4
    new_image = []
    new_image.append(empty_row(row_size, fill))
    new_image.append(empty_row(row_size, fill))
    for row in image:
        new_image.append([fill, fill, *row, fill, fill])
    new_image.append(empty_row(row_size, fill))
    new_image.append(empty_row(row_size, fill))
    return new_image


In [14]:
def get_new_character(algo: str, image: Image, i: int, j: int, fill: str) -> str:
    return algo[convert_binary(get_binary_pixel(image, i, j, fill))]

In [15]:
def get_fill_char(algo: str, image: Image, step: int):
    if algo[0] == ".":
        return "."
    else:
        return "." if (step % 2 == 0) else "#"

In [16]:
def update_image(algo: str, image: Image, step: int) -> Image:
    fill = get_fill_char(algo, image, step)
    resized_image = increase_image_size(image, fill)
    return [
        [
            get_new_character(algo, resized_image, i, j, fill) for j in range(len(resized_image[0]))
        ] for i in range(len(resized_image))
    ]

In [17]:
def get_light_count(algo: str, image: Image, steps: int) -> int:
    new_image = image
    for step in range(steps):
        new_image = update_image(algo, new_image, step)
    return count_light(new_image)

In [18]:
assert get_light_count(*demo_dataset, 2) == 35

In [19]:
get_light_count(*full_dataset, 2)

5479

# Part 2

In [20]:
get_light_count(*demo_dataset, 40)

2204

In [21]:
assert get_light_count(*demo_dataset, 50) == 3351

In [22]:
%%time
get_light_count(*full_dataset, 50)

CPU times: user 6.54 s, sys: 0 ns, total: 6.54 s
Wall time: 6.54 s


19012